hasta ahora hemos intentado predecir con una variable (`accommodates` y `bedrooms`)

hay variables que influyen en el precio

Hay varias formas de intentar mejorar la precisión
1. Cambiar el valor de k
2. Cambiar las variables (incrementar el número de atributos, p.ej, `accommodates`, `bedrooms`, `bathrooms`


Ahora lo que vamos a hacer es añadir columnas al modelo. Tenemos que tener cuidados con las variables que no van a funcionar con la ecuación de distancia:

- Valores no numéricos (categóricos) - la distancia quiere números
- Valores nulos: la distancia no permite nulos
- Valores no ordinales: numéricos pero que no representan un orden (ej.lat y la long) 

In [1]:
import pandas as pd
import numpy as np
import math


In [2]:
madrid = pd.read_csv('data/airbnb-madrid.csv')
# corrección de datos
madrid['price'] = madrid['price'].apply(lambda x: x.replace('$', '').replace(',', '')).astype(float)
np.random.seed(3943)
madrid = madrid.iloc[
    np.random.permutation(madrid.shape[0])
]


In [3]:
madrid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20681 entries, 682 to 15043
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            20681 non-null  int64  
 1   listing_url                                   20681 non-null  object 
 2   scrape_id                                     20681 non-null  int64  
 3   last_scraped                                  20681 non-null  object 
 4   source                                        20681 non-null  object 
 5   name                                          20677 non-null  object 
 6   description                                   19971 non-null  object 
 7   neighborhood_overview                         11472 non-null  object 
 8   picture_url                                   20680 non-null  object 
 9   host_id                                       20681 non-nul

In [4]:
madrid[
    ['bathrooms', 'bathrooms_text']
]

,bathrooms,bathrooms_text
682,NaN,1 bath
11870,NaN,1 bath
3526,NaN,2 baths
13270,NaN,1 bath
15946,NaN,1 bath
...,...,...
3351,NaN,1 bath
970,NaN,1.5 shared baths
4766,NaN,1.5 shared baths
4563,NaN,1 shared bath


In [13]:
madrid['bathrooms_text'].info()

<class 'pandas.core.series.Series'>
Int64Index: 20681 entries, 682 to 15043
Series name: bathrooms_text
Non-Null Count  Dtype 
--------------  ----- 
20657 non-null  object
dtypes: object(1)
memory usage: 323.1+ KB


In [17]:
def convert_half_in_number(x):
    if ('half' in str(x).lower()):
        return '0.5'
    else:
        return x
    
convert_half_in_number("single  batch")

'single  batch'

In [18]:
madrid['bathrooms_text'].apply(
    convert_half_in_number
)

1 bath              9620
1 shared bath       3748
2 baths             2585
1 private bath      1401
1.5 baths           1005
1.5 shared baths     691
2 shared baths       572
3 baths              293
2.5 baths            193
3 shared baths        88
4 shared baths        76
2.5 shared baths      58
4 baths               56
0 shared baths        52
3.5 baths             36
0 baths               35
0.5                   30
8 shared baths        24
5 baths               18
4.5 baths             18
5 shared baths        11
4.5 shared baths       9
6 baths                8
3.5 shared baths       8
6 shared baths         6
5.5 baths              4
7 baths                3
10 baths               2
6.5 baths              2
7 shared baths         2
8 baths                1
20 shared baths        1
5.5 shared baths       1
Name: bathrooms_text, dtype: int64

In [31]:
import re

def extract_number(x):
    return re.search("[0-9|.]*", str(x)).group(0)

In [39]:
madrid['bathrooms'] = madrid['bathrooms_text'].apply(
    convert_half_in_number
).apply(extract_number).apply(lambda x: '0' if str(x) == '' else x).astype('float')

In [43]:
madrid = madrid.dropna(axis= 1)

In [44]:
madrid['bathrooms'].describe()

count    20681.000000
mean         1.293965
std          0.648558
min          0.000000
25%          1.000000
50%          1.000000
75%          1.500000
max         20.000000
Name: bathrooms, dtype: float64

In [46]:
madrid['number_of_reviews'].describe()

count    20681.000000
mean        38.609207
std         73.588795
min          0.000000
25%          1.000000
50%          8.000000
75%         40.000000
max        845.000000
Name: number_of_reviews, dtype: float64

Distribución normal estándar : Media 0 y Desviación típica 1

Normalizar es:
 - restar a cada valor la media de la columna
 - dividir cada valor por la desviación típica
 

In [47]:
madrid['accommodates'].mean()

3.1138726367196945

In [48]:
madrid['accommodates'].std()

1.937425732532119

In [51]:
accommodates_normalized = (madrid['accommodates']-madrid['accommodates'].mean())/madrid['accommodates'].std()

In [50]:
madrid['accommodates']

682      4
11870    4
3526     4
13270    4
15946    4
        ..
3351     1
970      1
4766     2
4563     2
15043    2
Name: accommodates, Length: 20681, dtype: int64

In [52]:
accommodates_normalized.mean()

-2.88601082171309e-17

In [53]:
accommodates_normalized.std()


1.0